In [1]:
#take u.user and u.item into dataframes

import pandas as pd
import numpy as np

#u.user
users = pd.read_csv('./ml-100k/u.user', sep='|', names=['user_id' , 'age', 'gender', 'occupation', 'zip_code'])

#eliminamos las columnas que no necesitamos
#users = users.drop(['zip_code'], axis=1)


In [2]:
items = pd.read_csv('./ml-100k/u.item', sep='|', names=['movie_id', 'movie_title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], encoding='latin-1')


#eliminamos las columnas que no necesitamos
items = items.drop(['movie_title', 'release_date', 'video_release_date', 'IMDb_URL'], axis=1)

In [3]:
#lo mismo con u.data

ratings = pd.read_csv('./ml-100k/u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

#eliminamos la columna timestamp
ratings = ratings.drop(['timestamp'], axis=1)


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# Fusionamos los tres DataFrames en uno
data = pd.merge(pd.merge(ratings, users, on='user_id'), items, on='movie_id')
for genre in ['unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']:
    data[genre] *= data['rating']
data.drop(['movie_id', 'age','rating', 'zip_code', 'gender', 'occupation'], axis=1, inplace=True)

# Agrupamos por usuario y hacemos la media de las valoraciones
data = data.groupby('user_id').sum()

final_data = pd.merge(users[['user_id','age']].drop_duplicates(), data, on='user_id')






In [5]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcula la similitud del coseno entre cada par de usuarios
similarity_matrix = cosine_similarity(final_data)

# Crea un DataFrame a partir de la matriz de similitud
similarity_df = pd.DataFrame(similarity_matrix, index=final_data.index, columns=final_data.index)

similarity_df

,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,1.000000,0.904859,0.885925,0.826394,0.845158,0.944110,0.974739,0.801383,0.881599,0.952095,...,0.420885,0.127047,0.364972,0.107746,0.262211,0.162341,0.285114,0.070251,0.232599,0.401033
1,0.904859,1.000000,0.879815,0.786638,0.636943,0.951790,0.892971,0.661290,0.914882,0.959528,...,0.401992,0.139369,0.370448,0.129977,0.242917,0.159690,0.279895,0.067034,0.236021,0.363604
2,0.885925,0.879815,1.000000,0.955680,0.693516,0.840925,0.908733,0.799216,0.834644,0.911146,...,0.389229,0.148497,0.352443,0.126545,0.268654,0.172578,0.276092,0.091300,0.242609,0.392291
3,0.826394,0.786638,0.955680,1.000000,0.724694,0.719875,0.842368,0.851360,0.820030,0.811611,...,0.388388,0.171071,0.336674,0.140460,0.293831,0.193885,0.279810,0.121790,0.254978,0.401601
4,0.845158,0.636943,0.693516,0.724694,1.000000,0.709615,0.820153,0.751745,0.701889,0.709141,...,0.381349,0.112387,0.284995,0.088363,0.254069,0.152765,0.250131,0.085712,0.206917,0.367409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.162341,0.159690,0.172578,0.193885,0.152765,0.159270,0.163237,0.177161,0.257871,0.171597,...,0.958686,0.998687,0.972180,0.996373,0.993092,1.000000,0.990050,0.994910,0.993836,0.964222
939,0.285114,0.279895,0.276092,0.279810,0.250131,0.290250,0.280933,0.244913,0.367001,0.298388,...,0.986567,0.984181,0.993149,0.982174,0.994907,0.990050,1.000000,0.973073,0.995004,0.982027
940,0.070251,0.067034,0.091300,0.121790,0.085712,0.067667,0.075020,0.107405,0.170489,0.079618,...,0.929188,0.996219,0.945856,0.996014,0.979588,0.994910,0.973073,1.000000,0.982334,0.936177
941,0.232599,0.236021,0.242609,0.254978,0.206917,0.242640,0.243031,0.220962,0.330701,0.254786,...,0.976130,0.991507,0.983033,0.988639,0.996003,0.993836,0.995004,0.982334,1.000000,0.974953


In [6]:
#coger una fila aleatoria del dataframe ratings y predecir la valoracion de un usuario para una pelicula usando la similitud del coseno

# Elegimos una fila aleatoria del DataFrame ratings
random_user = ratings.sample(1)

# Obtenemos el ID del usuario
user_id = random_user['user_id'].values[0]

# Obtenemos el ID de la película
movie_id = random_user['movie_id'].values[0]

# Obtenemos la valoración del usuario para esa película
rating = random_user['rating'].values[0]

user_similarities = similarity_df[user_id]

# Filtramos ratings para incluir solo las filas donde movie_id es igual al de la película en cuestión
filtered_ratings = ratings[ratings['movie_id'] == movie_id].copy()

#restamos uno a la columna user_id para que coincida con el índice de similarity_df
filtered_ratings['user_id'] = filtered_ratings['user_id'] - 1

# Obtenemos las similitudes del usuario con los usuarios que han valorado la misma película
filtered_similarities = similarity_df.loc[user_id, filtered_ratings['user_id']]

# Obtenemos los índices de los 5 usuarios más similares
top_5_users = filtered_similarities.nlargest(6).index[1:]
# Obtenemos las valoraciones de los 5 usuarios más similares
top_5_user_ratings = filtered_ratings.loc[filtered_ratings['user_id'].isin(top_5_users), 'rating'].values
# Calculamos la media de las valoraciones
predicted_rating = top_5_user_ratings.mean()

#print predicted rating vs real rating
print('Predicted rating:', predicted_rating)
print('Real rating:', rating)


Predicted rating: 3.8
Real rating: 4


In [7]:
#Hacer 100 predicciones y calcular el MAE

mae = 0
contador = 0

for i in range(100):
    random_user = ratings.sample(1)
    user_id = random_user['user_id'].values[0] - 1
    movie_id = random_user['movie_id'].values[0]
    rating = random_user['rating'].values[0]
    user_similarities = similarity_df[user_id]

    filtered_ratings = ratings[ratings['movie_id'] == movie_id].copy()
    filtered_ratings['user_id'] = filtered_ratings['user_id'] - 1
    
    filtered_similarities = similarity_df.loc[user_id, filtered_ratings['user_id']]
    top_5_users = filtered_similarities.nlargest(6).index[1:]
    top_5_user_ratings = filtered_ratings.loc[filtered_ratings['user_id'].isin(top_5_users), 'rating'].values
    if top_5_user_ratings.size == 0:
        contador += 1
        continue
    
    predicted_rating = top_5_user_ratings.mean()
    mae += abs(predicted_rating - rating)
    
mae /= 100
print('Mean Absolute Error:', mae)
print('Number of predictions with no similar users:', contador)

Mean Absolute Error: 0.8980000000000001
Number of predictions with no similar users: 0


In [8]:
#abrir  u1.base y u1.test y hacer lo mismo que antes

train = pd.read_csv('./ml-100k/u1.base', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
test = pd.read_csv('./ml-100k/u1.test', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Fusionamos los tres DataFrames en uno
train_data = pd.merge(pd.merge(train, users, on='user_id'), items, on='movie_id')
for genre in ['unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']:
    train_data[genre] *= train_data['rating']
train_data.drop(['movie_id', 'age','rating', 'zip_code', 'gender', 'occupation'], axis=1, inplace=True)

train_data = data.groupby('user_id').sum()

final_train_data = pd.merge(users[['user_id','age']].drop_duplicates(), train_data, on='user_id')
print(final_train_data)
                 
                 

     user_id  age  unknown  Action  Adventure  Animation  Children  Comedy  \
0          1   24        4     250        123         40        55     316   
1          2   53        0      38         13          4        12      61   
2          3   23        0      39         14          0         0      31   
3          4   24        0      31         14          0         0      20   
4          5   33        4     176        107         53        71     246   
..       ...  ...      ...     ...        ...        ...       ...     ...   
938      939   26        0      73         35          4         4      68   
939      940   32        0      79         36          9        21     148   
940      941   20        0      38         27         14         9      29   
941      942   48        0      74         52         19        56      90   
942      943   22        0     227        114          7        25     166   

     Crime  Documentary  ...  Fantasy  Film-Noir  Horror  Music

In [9]:
# Calcula la similitud del coseno entre cada par de usuarios
similarity_matrix2 = cosine_similarity(final_train_data)

# Crea un DataFrame a partir de la matriz de similitud
similarity_df2 = pd.DataFrame(similarity_matrix2, index=final_train_data.index, columns=final_train_data.index)

similarity_df2

,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,1.000000,0.904859,0.885925,0.826394,0.845158,0.944110,0.974739,0.801383,0.881599,0.952095,...,0.420885,0.127047,0.364972,0.107746,0.262211,0.162341,0.285114,0.070251,0.232599,0.401033
1,0.904859,1.000000,0.879815,0.786638,0.636943,0.951790,0.892971,0.661290,0.914882,0.959528,...,0.401992,0.139369,0.370448,0.129977,0.242917,0.159690,0.279895,0.067034,0.236021,0.363604
2,0.885925,0.879815,1.000000,0.955680,0.693516,0.840925,0.908733,0.799216,0.834644,0.911146,...,0.389229,0.148497,0.352443,0.126545,0.268654,0.172578,0.276092,0.091300,0.242609,0.392291
3,0.826394,0.786638,0.955680,1.000000,0.724694,0.719875,0.842368,0.851360,0.820030,0.811611,...,0.388388,0.171071,0.336674,0.140460,0.293831,0.193885,0.279810,0.121790,0.254978,0.401601
4,0.845158,0.636943,0.693516,0.724694,1.000000,0.709615,0.820153,0.751745,0.701889,0.709141,...,0.381349,0.112387,0.284995,0.088363,0.254069,0.152765,0.250131,0.085712,0.206917,0.367409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.162341,0.159690,0.172578,0.193885,0.152765,0.159270,0.163237,0.177161,0.257871,0.171597,...,0.958686,0.998687,0.972180,0.996373,0.993092,1.000000,0.990050,0.994910,0.993836,0.964222
939,0.285114,0.279895,0.276092,0.279810,0.250131,0.290250,0.280933,0.244913,0.367001,0.298388,...,0.986567,0.984181,0.993149,0.982174,0.994907,0.990050,1.000000,0.973073,0.995004,0.982027
940,0.070251,0.067034,0.091300,0.121790,0.085712,0.067667,0.075020,0.107405,0.170489,0.079618,...,0.929188,0.996219,0.945856,0.996014,0.979588,0.994910,0.973073,1.000000,0.982334,0.936177
941,0.232599,0.236021,0.242609,0.254978,0.206917,0.242640,0.243031,0.220962,0.330701,0.254786,...,0.976130,0.991507,0.983033,0.988639,0.996003,0.993836,0.995004,0.982334,1.000000,0.974953


In [10]:
# Elegimos una fila aleatoria del DataFrame test
random_user = test.sample(1)

# Obtenemos el ID del usuario
user_id = random_user['user_id'].values[0]

# Obtenemos el ID de la película
movie_id = random_user['movie_id'].values[0]

# Obtenemos la valoración del usuario para esa película
rating = random_user['rating'].values[0]

user_similarities = similarity_df[user_id]

# Filtramos ratings para incluir solo las filas donde movie_id es igual al de la película en cuestión
filtered_ratings = ratings[ratings['movie_id'] == movie_id].copy()

#restamos uno a la columna user_id para que coincida con el índice de similarity_df
filtered_ratings['user_id'] = filtered_ratings['user_id'] - 1

# Obtenemos las similitudes del usuario con los usuarios que han valorado la misma película
filtered_similarities = similarity_df.loc[user_id, filtered_ratings['user_id']]
# Obtenemos los índices de los 5 usuarios más similares
top_5_users = filtered_similarities.nlargest(5).index[:]
# Obtenemos las valoraciones de los 5 usuarios más similares
top_5_user_ratings = filtered_ratings.loc[filtered_ratings['user_id'].isin(top_5_users), 'rating'].values
# Calculamos la media de las valoraciones
predicted_rating = top_5_user_ratings.mean()

#print predicted rating vs real rating
print('Predicted rating:', predicted_rating)
print('Real rating:', rating)

Predicted rating: 3.8
Real rating: 3


In [11]:
# Hacer las prediccion con el conjunto de test y calcular el MAE

mae = 0
cont = 0

for i in range(len(test)):
    random_user = test.iloc[i]
    user_id = random_user['user_id'] - 1
    movie_id = random_user['movie_id']
    rating = random_user['rating']
    user_similarities = similarity_df2[user_id]
    
    filtered_ratings = train[train['movie_id'] == movie_id].copy()
    filtered_ratings['user_id'] = filtered_ratings['user_id'] - 1
    
    filtered_similarities = similarity_df2.loc[user_id, filtered_ratings['user_id']]
    top_5_users = filtered_similarities.nlargest(5).index[:]
    top_5_user_ratings = filtered_ratings.loc[filtered_ratings['user_id'].isin(top_5_users), 'rating'].values
    if top_5_user_ratings.size == 0:
        #numero aleatorio entre 1 y 5
        predicted_rating = np.random.randint(1, 6)
        mae += abs(predicted_rating - rating)
        cont += 1
        continue
    
    predicted_rating = top_5_user_ratings.mean()
    mae += abs(predicted_rating - rating)
    
mae /= len(test)
print('Mean Absolute Error:', mae)
print('Number of predictions with no similar users:', cont)



Mean Absolute Error: 0.8618200000000237
Number of predictions with no similar users: 32


In [16]:
# Haer lo mismo pero entrenando una SVM con los 5 mas similares

from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
mae = []

# iterando sobre usuarios

users = test['user_id'].unique()

for user_id in users:
    train_user = train[train['user_id'] == user_id]
    test_user = test[test['user_id'] == user_id]

    # usuarios mas similares
    #user_similarities = similarity_df2[user_id - 1]
    #top_5_users = user_similarities.nlargest(5).index[:]
    #top_5_users = top_5_users + 1

    # coger filas del train con los usuarios mas similares
    #train_filtered = train[train['user_id'].isin(top_5_users)]
    
    # concatenar train_user y train_filtered
    #train_data = pd.concat([train_user, train_filtered])
 
    # coger las columnas que nos interesan
    train_data = train_user[['user_id', 'movie_id', 'rating']]

    # coger las columnas que nos interesan
    test_data = test_user[['user_id', 'movie_id', 'rating']]

    # ENTRENAR SVM CON columan rating como target
    X_train = train_data[['user_id', 'movie_id']]
    y_train = train_data['rating']
    X_test = test_data[['user_id', 'movie_id']]
    y_test = test_data['rating']

    svr = SVR()
    svr.fit(X_train, y_train)
    predictions = svr.predict(X_test)
    mae.append(mean_absolute_error(y_test, predictions))

mae = np.mean(mae)

print('Mean Absolute Error:', mae)


Mean Absolute Error: 0.8377677295592922


In [12]:
# Calcular las similitudes con KNN de surprise

from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train[['user_id', 'movie_id', 'rating']], reader)

sim_options = {
    'name': 'cosine',
    'user_based': True
}

knn = KNNBasic(sim_options=sim_options)
# Hacer las predicciones con KNN de surprise

knn.fit(data.build_full_trainset())

mae = 0
cont = 0

for i in range(len(test)):
    random_user = test.iloc[i]
    user_id = random_user['user_id']
    movie_id = random_user['movie_id']
    rating = random_user['rating']
    
    predicted_rating = knn.predict(user_id, movie_id).est
    mae += abs(predicted_rating - rating)

mae /= len(test)
print('Mean Absolute Error:', mae)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Mean Absolute Error: 0.8127633200701622


In [14]:
from sklearn.svm import SVR

#Introducir este knn en el coidigo de la SVM a la hora de encotnrar los usuarios mas similares

mae = []

users = test['user_id'].unique()

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(train[['user_id', 'movie_id', 'rating']], reader)

sim_options = {
    'name': 'cosine',
    'user_based': True
}

knn = KNNBasic(sim_options=sim_options)
# Hacer las predicciones con KNN de surprise

knn.fit(data.build_full_trainset())

for user_id in users:
    train_user = train[train['user_id'] == user_id]
    test_user = test[test['user_id'] == user_id]

    # usuarios mas similares con KNN de surprise
    user_similarities = knn.get_neighbors(user_id, 5)
    top_5_users = user_similarities 

    # coger filas del train con los usuarios mas similares
    train_filtered = train[train['user_id'].isin(top_5_users)]
    
    # concatenar train_user y train_filtered
    train_data = pd.concat([train_user, train_filtered])

    # coger las columnas que nos interesan
    train_data = train_data[['user_id', 'movie_id', 'rating']]

    # coger las columnas que nos interesan
    test_data = test_user[['user_id', 'movie_id', 'rating']]

    # ENTRENAR SVM CON columan rating como target
    X_train = train_data[['user_id', 'movie_id']]
    y_train = train_data['rating']
    X_test = test_data[['user_id', 'movie_id']]
    y_test = test_data['rating']

    svr = SVR()
    svr.fit(X_train, y_train)
    predictions = svr.predict(X_test)
    mae.append(np.abs(predictions - y_test).mean())

mae = np.mean(mae)

print('Mean Absolute Error:', mae)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Mean Absolute Error: 0.8417027946066701
